In [59]:
import nltk
nltk.download("brown")
nltk.download("webtext")
nltk.download("reuters")
nltk.download("punkt")
from nltk.corpus import brown, webtext, reuters
brown_corpus = brown.sents()
brown_corpus = [" ".join(sentence) for sentence in brown_corpus]
brown_corpus = ["<s> " + sentence + " </s>" for sentence in brown_corpus][:5000]
webtext_corpus = webtext.sents()
webtext_corpus = [" ".join(sentence) for sentence in webtext_corpus]
webtext_corpus = ["<s> " + sentence + " </s>" for sentence in webtext_corpus][:5000]
reuters_corpus = reuters.sents()
reuters_corpus = [" ".join(sentence) for sentence in reuters_corpus]
reuters_corpus = ["<s> " + sentence + " </s>" for sentence in reuters_corpus][:5000]

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\ruban\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package webtext to
[nltk_data]     C:\Users\ruban\AppData\Roaming\nltk_data...
[nltk_data]   Package webtext is already up-to-date!
[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\ruban\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ruban\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [60]:
def generate_unigrams(text):
    return text.split(" ")

In [61]:
# Function to extract unigrams, bigrams, and trigrams from a corpus
def extract_unigrams(corpus):
    unigrams = []
    for sentence in corpus:
        unigrams.extend(generate_unigrams(sentence))
    return unigrams

def extract_bigrams(unigrams):
    return list(zip(unigrams, unigrams[1:]))

def extract_trigrams(unigrams):
    return list(zip(unigrams, unigrams[1:], unigrams[2:]))

In [62]:
# Generating n-grams for Brown corpus
unigram_brown = extract_unigrams(brown_corpus[:25])
bigram_brown = extract_bigrams(unigram_brown[:25])
trigram_brown = extract_trigrams(unigram_brown[:25])

# Generating n-grams for Webtext corpus
unigram_webtext = extract_unigrams(webtext_corpus[:25])
bigram_webtext = extract_bigrams(unigram_webtext[:25])
trigram_webtext = extract_trigrams(unigram_webtext[:25])

In [63]:
bigram_brown

[('<s>', 'The'),
 ('The', 'Fulton'),
 ('Fulton', 'County'),
 ('County', 'Grand'),
 ('Grand', 'Jury'),
 ('Jury', 'said'),
 ('said', 'Friday'),
 ('Friday', 'an'),
 ('an', 'investigation'),
 ('investigation', 'of'),
 ('of', "Atlanta's"),
 ("Atlanta's", 'recent'),
 ('recent', 'primary'),
 ('primary', 'election'),
 ('election', 'produced'),
 ('produced', '``'),
 ('``', 'no'),
 ('no', 'evidence'),
 ('evidence', "''"),
 ("''", 'that'),
 ('that', 'any'),
 ('any', 'irregularities'),
 ('irregularities', 'took'),
 ('took', 'place')]

In [64]:
# Function to count token frequencies
def count_frequencies(tokens):
    return Counter(tokens)

# Function to count n-gram frequencies
def count_ngram_frequencies(ngrams):
    return Counter(ngrams)

# Function to get count of a specific n-gram
def get_ngram_count(ngram, ngram_list):
    return ngram_list.count(ngram)

In [65]:
# Conditional probability for bigrams
def bigram_probability(w1, w2, bigram_list, unigram_list):
    bigram = (w1, w2)
    bigram_count = get_ngram_count(bigram, bigram_list)
    unigram_count = unigram_list.count(w1)
    return (bigram_count + 1) / (unigram_count + len(set(unigram_list)))

# Conditional probability for trigrams
def trigram_probability(w1, w2, w3, trigram_list, bigram_list, unigram_list):
    trigram = (w1, w2, w3)
    trigram_count = get_ngram_count(trigram, trigram_list)
    bigram_count = get_ngram_count((w1, w2), bigram_list)
    return (trigram_count + 1) / (bigram_count + len(set(unigram_list)))

In [66]:
# Function to calculate bigram perplexity
def bigram_perplexity(sentence, unigrams, bigrams):
    words = generate_unigrams(sentence)
    unigram_counts = Counter(unigrams)
    N = len(words)
    prob = unigram_counts[words[0]] / N
    for i in range(1, N):
        prob *= bigram_probability(words[i-1], words[i], bigrams, unigrams)
    return (1 / prob) ** (1 / N)

In [67]:
# Function to calculate trigram perplexity
def trigram_perplexity(sentence, unigrams, bigrams, trigrams):
    words = generate_unigrams(sentence)
    N = len(words)
    prob = bigram_probability(words[0], words[1], bigrams, unigrams)
    for i in range(2, N):
        prob *= trigram_probability(words[i-2], words[i-1], words[i], trigrams, bigrams, unigrams)
    return (1 / prob) ** (1 / N)

In [75]:
# Compute bigram perplexity for first 5 Brown corpus sentences
perplexity = sum(bigram_perplexity(sent, unigram_brown, bigram_brown) for sent in brown_corpus[:5])
print("bigram perplexity for first 5 Brown corpus sentences",perplexity / 5)

bigram perplexity for first 5 Brown corpus sentences 241.52576419085705


In [76]:
# Compute trigram perplexity for first 5 Brown corpus sentences
perplexity = sum(trigram_perplexity(sent, unigram_brown, bigram_brown, trigram_brown) for sent in brown_corpus[:5])
print("trigram perplexity for first 5 Brown corpus sentences",perplexity / 5)

trigram perplexity for first 5 Brown corpus sentences 231.80852210203474


In [74]:
# Compute bigram perplexity for first 25 Reuters corpus sentences using Webtext model
perplexity = sum(bigram_perplexity(sent, unigram_webtext, bigram_webtext) for sent in reuters_corpus[:25])
print("bigram perplexity for first 25 Reuters corpus sentences using Webtext model",perplexity / 25)

bigram perplexity for first 25 Reuters corpus sentences using Webtext model 664.6676696435447


In [73]:
# Compute bigram perplexity for first 25 Reuters corpus sentences using Brown model
perplexity = sum(bigram_perplexity(sent, unigram_brown, bigram_brown) for sent in reuters_corpus[:25])
print("bigram perplexity for first 25 Reuters corpus sentences using Brown model",perplexity / 25)

bigram perplexity for first 25 Reuters corpus sentences using Brown model 253.5422379735836
